In [33]:
from signals import *
import pandas as pd
import numpy as np
from indicators.indicator.indicators import SMA, SuperTrend
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

In [45]:
def meta_super_trend(df, sample_mins, period, multiplier):
    ohlc_dict = {'Open':'first', 'High':'max', 'Low':'min', 'Close': 'last'}
    interval = (df.index[1]-df.index[0]).components
    base_interval = interval.hours * 60 + interval.minutes
    dfs = []
    for base in range(0, sample_mins, base_interval):
        tmp = df.resample(f"{sample_mins}Min", base=base).apply(ohlc_dict).dropna()
        SuperTrend(tmp, period, multiplier, prefix=sample_mins)
        dfs.append(tmp)

    return pd.concat(dfs).sort_index()

In [54]:
df = load_ticker("EURUSD", 60)
df = meta_super_trend(df, 240, 50, 2)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(df)

In [56]:
dfp = df.loc["2019-05-02 09:00:00":"2019-06-05 09:00:00"]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.layout.xaxis.type = 'category'
fig.update_layout(
    autosize=False,
    width=900,
    height=1000
)

fig.add_trace(go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close']))

fig.add_trace(go.Scatter(x=dfp.index, y=dfp["240_ST_50_2"],
                    mode='lines',
                    name='lines'),
                    secondary_y=True)